In [9]:
from sentence_transformers import SentenceTransformer, util

In [8]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [11]:
sentences = ["The weather today is beautiful", "It's raining!", "Dogs are awesome"]
embeddings = model.encode(sentences)
query_embedding = model.encode("today is sunny day!")

for sentence, emb in zip(sentences, embeddings):
    score = util.pytorch_cos_sim(query_embedding, emb)
    print(f"score: {score}", sentence)

score: tensor([[0.6973]]) The weather today is beautiful
score: tensor([[0.5133]]) It's raining!
score: tensor([[0.1889]]) Dogs are awesome


In [12]:
faq = {
    "How do I get a replacement Medicare card?": "If your Medicare card was lost, stolen, or destroyed, you can request a replacement online at Medicare.gov.",
    "How do I sign up for Medicare?": "If you already get Social Security benefits, you do not need to sign up for Medicare. We will automatically enroll you in Original Medicare (Part A and Part B) when you become eligible. We will mail you the information a few months before you become eligible.",
    "What are Medicare late enrollment penalties?": "In most cases, if you don’t sign up for Medicare when you’re first eligible, you may have to pay a higher monthly premium. Find more information at https://faq.ssa.gov/en-us/Topic/article/KA-02995",
    "Will my Medicare premiums be higher because of my higher income?": "Some people with higher income may pay a larger percentage of their monthly Medicare Part B and prescription drug costs based on their income. We call the additional amount the income-related monthly adjustment amount.",
    "What is Medicare and who can get it?": "Medicare is a health insurance program for people age 65 or older. Some younger people are eligible for Medicare including people with disabilities, permanent kidney failure and amyotrophic lateral sclerosis (Lou Gehrig’s disease or ALS). Medicare helps with the cost of health care, but it does not cover all medical expenses or the cost of most long-term care.",
}

In [18]:
corpus_embeddings = model.encode(list(faq.keys()))
query_emb = model.encode('do i have to pay more after my raise?')
similarities = util.semantic_search(query_emb, corpus_embeddings, top_k=3)

for i, res in enumerate(similarities[0]):
    id, score = res['corpus_id'], res['score']
    query =  list(faq.keys())[id]
    answer = faq[query]
    print(f"score:{score:.2f}, query: {query}, answer: {answer}\n")

score:0.46, query: Will my Medicare premiums be higher because of my higher income?, answer: Some people with higher income may pay a larger percentage of their monthly Medicare Part B and prescription drug costs based on their income. We call the additional amount the income-related monthly adjustment amount.

score:0.14, query: What is Medicare and who can get it?, answer: Medicare is a health insurance program for people age 65 or older. Some younger people are eligible for Medicare including people with disabilities, permanent kidney failure and amyotrophic lateral sclerosis (Lou Gehrig’s disease or ALS). Medicare helps with the cost of health care, but it does not cover all medical expenses or the cost of most long-term care.

score:0.13, query: What are Medicare late enrollment penalties?, answer: In most cases, if you don’t sign up for Medicare when you’re first eligible, you may have to pay a higher monthly premium. Find more information at https://faq.ssa.gov/en-us/Topic/artic